In [11]:
import re
import numpy as np

In [12]:
# open file from folder

file_path = "DUC_TEXT/test/d112h"  

with open(file_path, "r", encoding="utf-8") as file:
    doc_file = file.read()

#print(doc_file)

# parsing docs to doc_list 

In [13]:
def parse_docs(docs_string):
    doc_list = []
    # Find all sentence tags
    sentences = re.findall(r'<s.*?</s>', docs_string, re.DOTALL)

    for sentence in sentences:
        doc_info = {}
        # Extract docid, num, and wdcount from the tag
        match = re.search(r'<s docid="(.*?)" num="(.*?)" wdcount="(.*?)">', sentence)
        if match:
            doc_info['docid'] = match.group(1)
            doc_info['num'] = int(match.group(2))
            doc_info['wdcount'] = int(match.group(3))

            # Extract the text content within the tag
            text = re.sub(r'<s.*?">', '', sentence)
            text = re.sub(r'</s>', '', text).strip()
            doc_info['full_text'] = text

            doc_list.append(doc_info)

    return doc_list


In [14]:
# Create the list of dictionaries
docs_data = parse_docs(doc_file)

# Print the resulting list of dictionaries
#print(docs_data)

# If you want a dictionary where keys are docid-num combinations
docs_dict = {}
for doc in docs_data:
    key = f"{doc['num']}"
    docs_dict[key] = {
        'docid': doc['docid'],

        'wdcount': doc['wdcount'],
        'full_text': doc['full_text']
    }

# Print the resulting dictionary
#print(docs_dict)

compare full text to each other, if they have n (for ex. 4, can change the number later for testing) common words then say they have an connection, save the index of the sentence and the connection btw two sentences as boolean  to an balanced two-dimensional array,
 """
    Checks if two strings have a connection based on common words.

    Args:
        text1 (str): The first string.
        text2 (str): The second string.
        min_common_words (int): The minimum number of common words required for a connection.

    Returns:
        bool: True if the strings have a connection, False otherwise.
    """

In [15]:
def has_connection(text1, text2, min_common_words=7):
   
    words1 = set(re.findall(r'\b\w+\b', text1.lower()))
    words2 = set(re.findall(r'\b\w+\b', text2.lower()))
    common_words = words1.intersection(words2)
    return len(common_words) >= min_common_words

# Get the number of sentences
num_sentences = len(docs_dict)

# Initialize a balanced two-dimensional array (matrix) with False
connection_matrix = np.full((num_sentences, num_sentences), False, dtype=bool)

# Iterate through all pairs of sentences to check for connections
sentence_keys = list(docs_dict.keys())
for i in range(num_sentences):
    for j in range(num_sentences):
        if i != j:  # Avoid comparing a sentence to itself
            text1 = docs_dict[sentence_keys[i]]['full_text']
            text2 = docs_dict[sentence_keys[j]]['full_text']
            if has_connection(text1, text2):
                connection_matrix[i, j] = True

# Print the connection matrix
# print("\nConnection Matrix:")
# print(connection_matrix)

# You can also print which sentences are connected
# print("\nSentence Connections:")
# for i in range(num_sentences):
#     for j in range(num_sentences):
#         if connection_matrix[i, j]:
#             print(f"Sentence {sentence_keys[i]} is connected to Sentence {sentence_keys[j]}")

# PageRank Formula

# The PageRank formula is defined as:

$$
PR(i) = \frac{1 - d}{N} + d \sum_{j \in M(i)} \frac{PR(j)}{L(j)}
$$

Where:

- $PR(i)$ is the PageRank of node $i$
- $d$ is the damping factor (usually 0.85)
- $N$ is the total number of nodes
- $M(i)$ are nodes linking to $i$
- $L(j)$ is the number of outgoing links from node $j$

In [16]:
def pagerank(matrix, damping_factor=0.85, max_iterations=5, tolerance=1e-6):
    """
    Calculates the PageRank score for each node in a graph.

    Args:
        matrix (np.array): The adjacency matrix of the graph (boolean or integer).
                           matrix[i, j] is True if there is a link from node i to node j.
        damping_factor (float): The damping factor (usually 0.85).
        max_iterations (int): Maximum number of iterations.
        tolerance (float): Convergence tolerance.

    Returns:
        np.array: A numpy array containing the PageRank score for each node.
    """
    num_nodes = matrix.shape[0]

    # Convert boolean matrix to float and normalize rows
    # Create the transition matrix
    transition_matrix = matrix.astype(float)
    row_sums = transition_matrix.sum(axis=1)
    # Handle nodes with no outgoing links (dangling nodes)
    # Add links to all other nodes, distributing their weight evenly
    dangling_nodes = np.where(row_sums == 0)[0]
    for node in dangling_nodes:
        transition_matrix[node, :] = 1.0 / num_nodes
    # Normalize non-dangling rows
    non_dangling_rows = np.where(row_sums != 0)[0]
    transition_matrix[non_dangling_rows, :] /= row_sums[non_dangling_rows, None]

    # Initialize PageRank vector
    pr = np.ones(num_nodes) / num_nodes

    # Iteratively calculate PageRank
    for iteration in range(max_iterations):
        prev_pr = pr.copy()
        # PageRank formula: PR(i) = (1-d)/N + d * sum(PR(j)/L(j)) for all j linking to i
        # In matrix form: PR = (1-d)/N * 1 + d * M.T @ PR  where M is the transition matrix
        pr = (1 - damping_factor) / num_nodes + damping_factor * (transition_matrix.T @ pr)

        # Check for convergence
        if np.linalg.norm(pr - prev_pr, ord=1) < tolerance:
            print(f"PageRank converged after {iteration + 1} iterations.")
            break
    else:
        print(f"PageRank did not converge after {max_iterations} iterations.")

    return pr

# Calculate PageRank for the sentences using the connection matrix
sentence_pagerank_scores = pagerank(connection_matrix)

# Print the PageRank scores
# print("\nPageRank Scores for Sentences:")
# sentence_keys = list(docs_dict.keys())
# for i, score in enumerate(sentence_pagerank_scores):
#     print(f"Sentence {sentence_keys[i]}: {score:.6f}")

PageRank did not converge after 5 iterations.


sort the score decendant and print out 10 of highest score snetences

In [ ]:
# Combine sentence keys and scores into a list of tuples
sentence_scores = list(zip(sentence_keys, sentence_pagerank_scores))

# Sort the sentences based on their PageRank scores in descending order
sorted_sentences = sorted(sentence_scores, key=lambda item: item[1], reverse=True)

# Print the 10 highest scoring sentences
print("\nTop 10 Highest Scoring Sentences (by PageRank):")
for i, (key, score) in enumerate(sorted_sentences[:10]):
    print(f"{i+1}. Sentence {key} (Score: {score:.6f}):")
    print(docs_dict[key]['full_text'])
    print("-" * 20)

In [18]:
import os

# Create output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Get the input file name without the extension
input_filename = os.path.splitext(os.path.basename(file_path))[0]

# Define the output file path
output_file_path = os.path.join(output_dir, f'{input_filename}_top_sentences.txt')

# Write the top sentences to the output file in the desired format
with open(output_file_path, 'w') as outfile:
    for key, score in sorted_sentences[:10]:
        docid = docs_dict[key]['docid']
        wdcount = docs_dict[key]['wdcount']
        full_text = docs_dict[key]['full_text']
        # Reconstruct the original sentence tag format
        outfile.write(f'<s docid="{docid}" num="{key}" wdcount="{wdcount}"> {full_text}</s>\n')

print(f"\nTop 10 sentences written to {output_file_path}")


Top 10 sentences written to output/d112h_top_sentences.txt
